#  Text Clssification Practical


I this we get a dataset and apply all type of data preporessing techniques and after all i extract features from the data after all use machine learning model for procesing 

In [3]:
import kagglehub
import os
import shutil

# Step 1: Download using kagglehub
download_path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Original path:", download_path)

# Step 2: Move files to current directory
current_dir = os.getcwd()

# Move all files inside the downloaded folder
for filename in os.listdir(download_path):
    src = os.path.join(download_path, filename)
    dst = os.path.join(current_dir, filename)
    shutil.copy2(src, dst)  # use copy2 to preserve metadata

print("✅ Dataset copied to:", current_dir)


Original path: C:\Users\Mehboob Alam\.cache\kagglehub\datasets\lakshmi25npathi\imdb-dataset-of-50k-movie-reviews\versions\1
✅ Dataset copied to: m:\Gen AI\Projects\Text Classification


In [3]:
%pip install keras

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re 
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer




from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix


pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',255)

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package stopwords to C:\Users\Mehboob
[nltk_data]     Alam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
path="IMDB Dataset.csv"
df=pd.read_csv(path)
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of v...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen-...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well b...",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br...",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situ...",positive


In [5]:
df.shape

(50000, 2)

In [6]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [7]:
# Take the 10k rows for training
df = df.iloc[:10000]

In [8]:
df.shape

(10000, 2)

In [9]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [10]:
# check the duplicate daata of rows in my database 
df.duplicated().sum()

17

In [11]:
#remove the duplicate data from my database
df.drop_duplicates(inplace=True)

In [12]:
#check the duplicate daata of rows in my database after remove its 
df.duplicated().sum()

0

In [13]:
df.shape

(9983, 2)

In [14]:
#check the unique values count  in sentinmentt
df['sentiment'].value_counts()

sentiment
positive    5023
negative    4960
Name: count, dtype: int64

In [15]:
# check null values in rows 
df.isnull().sum()

review       0
sentiment    0
dtype: int64

## Basic Preprocessing 

- convert into Lowercase
- Remove Html tags 
- Remove the stopwords 


In [16]:
# convert into lower case 
def convert_lowercase(text):
    return text.lower()


In [17]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [18]:
df['review']=df['review'].apply(convert_lowercase)

In [19]:
df['review'][0] # check after apply the function of lowercase conversion

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.<br /><br />the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.<br /><br />it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />i would say the main appeal of the show is due to the fa

In [20]:
# Remove HTML tags
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


In [21]:
df['review'][0] # before removing the html tags 

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.<br /><br />the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.<br /><br />it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />i would say the main appeal of the show is due to the fa

In [22]:
df['review']=df['review'].apply(remove_html_tags)

In [23]:
df['review'][0]  #after removing the html tags

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [24]:
# remove the stop words
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

In [25]:
df['review'][0] # before removing  stops words 

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [26]:
df['review']=df['review'].apply(remove_stopwords)

In [27]:
df['review'][0] #after removing the stop words

"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. hardcore, classic use word.it called oz nickname given oswald maximum security state penitentary. focuses mainly emerald city, experimental section prison cells glass fronts face inwards, privacy high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealings shady agreements never far away.i would say main appeal show due fact goes shows dare. forget pretty pictures painted mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw struck nasty surreal, say ready it, watched more, developed taste oz, got accustomed high levels graphic violence. violence, injustice (crooked guards who'll sold nickel, inmates who'll kill order g

### Split the data for model

In [28]:

X = df['review']
Y = df['sentiment']

In [29]:
X

0       one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. ha...
1       wonderful little production. filming technique unassuming- old-time-bbc fashion gives comforting, sometimes discomforting, sense realism entire piece. actors extremely well chosen- michael sheen "has got polari" voices pat too! truly see seamless edit...
2       thought wonderful way spend time hot summer weekend, sitting air conditioned theater watching light-hearted comedy. plot simplistic, dialogue witty characters likable (even well bread suspected serial killer). may disappointed realize match point 2: r...
3       basically there's family little boy (jake) thinks there's zombie closet & parents fighting time.this movie slower soap opera... suddenly, jake decides become rambo kill zombie.ok, first going make film m

In [30]:
Y

0       positive
1       positive
2       positive
3       negative
4       positive
5       positive
6       positive
7       negative
8       negative
9       positive
10      negative
11      negative
12      negative
13      negative
14      positive
15      negative
16      positive
17      negative
18      positive
19      negative
20      positive
21      negative
22      positive
23      negative
24      negative
25      positive
26      positive
27      negative
28      negative
29      positive
30      positive
31      positive
32      negative
33      positive
34      negative
35      negative
36      negative
37      negative
38      positive
39      negative
40      negative
41      positive
42      negative
43      negative
44      positive
45      positive
46      negative
47      negative
48      positive
49      negative
50      positive
51      positive
52      positive
53      positive
54      negative
55      negative
56      negative
57      negative
58      positi

In [31]:
# now convert thhe Y label data into numerical values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [32]:
Y= le.fit_transform(Y)

In [33]:
Y

array([1, 1, 1, ..., 0, 0, 1])

### split Data for Training and Testing 

In [34]:
from sklearn.model_selection import train_test_split
# split Data for Training and Testing

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [35]:
X_train.shape

(7986,)

In [36]:
X_test.shape

(1997,)

### Apply BOW

In [70]:
# apply bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [77]:
X_train_BOW=cv.fit_transform(X_train).toarray()
X_test_BOW=cv.transform(X_test).toarray()

In [78]:
X_train_BOW

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [80]:
X_test_BOW

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## Apply ML model Naive Bayes

In [81]:
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()

gnb.fit(X_train_BOW, Y_train)

,priors,None
,var_smoothing,1e-09


In [82]:
y_prect= gnb.predict(X_test_BOW)

from sklearn.metrics import accuracy_score,confusion_matrix
accuracy = accuracy_score(Y_test, y_prect)
print("Accuracy:", accuracy)    

Accuracy: 0.6324486730095142


In [83]:
confusion_matrix = confusion_matrix(Y_test, y_prect)
print("Confusion Matrix:\n", confusion_matrix)

Confusion Matrix:
 [[717 235]
 [499 546]]


### Apply the Rnadom Forest Classifier

In [84]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

rfc.fit(X_train_BOW, Y_train)

y_prect_rfc = rfc.predict(X_test_BOW)

accuracy_rfc = accuracy_score(Y_test, y_prect_rfc)
print("Random Forest Classifier Accuracy:", accuracy_rfc)

Random Forest Classifier Accuracy: 0.8432648973460191


### I fixed the max features(unique words table) to 3000 in BOW

In [86]:
cv_3000=CountVectorizer(max_features=3000)

X_train_BOW_3000 = cv_3000.fit_transform(X_train).toarray()
X_test_BOW_3000 = cv_3000.transform(X_test).toarray()

In [87]:
rfc_3000 = RandomForestClassifier()
rfc_3000.fit(X_train_BOW_3000, Y_train)

y_prect_rfc_3000 = rfc_3000.predict(X_test_BOW_3000)

In [88]:
accuracy_rfc_3000 = accuracy_score(Y_test, y_prect_rfc_3000)
print("Random Forest Classifier with 3000 features Accuracy:", accuracy_rfc_3000)

Random Forest Classifier with 3000 features Accuracy: 0.8397596394591887


# Ngram 

In [89]:
cv_ngram = CountVectorizer(ngram_range=(2, 2),max_features=5000)

X_train_ngram = cv_ngram.fit_transform(X_train).toarray()
X_test_ngram = cv_ngram.transform(X_test).toarray()

In [92]:
rfc_ngram = RandomForestClassifier()
rfc_ngram.fit(X_train_ngram, Y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [93]:
y_prect_ngram = rfc_ngram.predict(X_test_ngram)
accuracy_ngram = accuracy_score(Y_test, y_prect_ngram)
print("Random Forest Classifier with N-grams Accuracy:", accuracy_ngram)

Random Forest Classifier with N-grams Accuracy: 0.7531296945418127


# TF IDF 

In [94]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_BOW)
X_test_tfidf = tfidf_transformer.transform(X_test_BOW)



In [95]:
rfc_tfidf = RandomForestClassifier()
rfc_tfidf.fit(X_train_tfidf, Y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [97]:
accuracy_tfidf = accuracy_score(Y_test, rfc_tfidf.predict(X_test_tfidf))
print("Random Forest Classifier with TF-IDF Accuracy:", accuracy_tfidf)

Random Forest Classifier with TF-IDF Accuracy: 0.842764146219329


In [38]:
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Tokenize reviews for Word2Vec
X_train_tokens = [review.split() for review in X_train]
X_test_tokens = [review.split() for review in X_test]

# Train Word2Vec model
w2v_model = Word2Vec(sentences=X_train_tokens)

# Function to get averaged word vectors for a review
def get_review_vector(tokens, model, vector_size):
    vec = np.zeros(vector_size)
    count = 0
    for word in tokens:
        if word in model.wv:
            vec += model.wv[word]
            count += 1
    if count > 0:
        vec /= count
    return vec

# Transform train and test sets
X_train_w2v = np.array([get_review_vector(tokens, w2v_model, 100) for tokens in X_train_tokens])
X_test_w2v = np.array([get_review_vector(tokens, w2v_model, 100) for tokens in X_test_tokens])

# Train Random Forest on Word2Vec features
rfc_w2v = RandomForestClassifier()
rfc_w2v.fit(X_train_w2v, Y_train)

y_prect_w2v = rfc_w2v.predict(X_test_w2v)
accuracy_w2v = accuracy_score(Y_test, y_prect_w2v)
print("Random Forest Classifier with Word2Vec Accuracy:", accuracy_w2v)

Random Forest Classifier with Word2Vec Accuracy: 0.7275913870806209
